In [1]:
ipython = get_ipython()
colab   = 'google.colab' in str(ipython)
print(ipython)

if colab:
  from google.colab import drive
  drive.mount('/content/gdrive')
  %cd '/content/gdrive/My Drive/Colab Notebooks'

!ls "./"

def MODULE_FROM_NOTEBOOK(target, source, *keys): 
    from json import load
    with open(source) as notebook:
        data = load(notebook)
        with open(target,'w') as module:
          for cell in data['cells']:
              lines = cell['source']
              if type(lines) == str:
                  lines = lines.split('\n')
              line0  = (lines or [''])[0].strip()
              haskey = lambda key: line0 == f'### {key} ###'
              if cell['cell_type'] == 'code' and any(map(haskey,keys)):
                  code = '\n'.join(lines)
                  module.write(code)
                  module.write('\n')
                     
MODULE_FROM_NOTEBOOK('./ttt_tactics.py',
                     './ttt-tactics.ipynb',
                     'HEAD','GAME','DATA','MODEL','SEARCH')

from ttt_tactics import *

ipynb = 'ttt-tactics-play'


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Colab Notebooks
'!cpu.ipynb'   ttt-tactics	        ttt-tactics-test.ipynb
'!gpu.ipynb'   ttt-tactics.ipynb        ttt-tactics-train.ipynb
 __pycache__   ttt-tactics-play.ipynb
 ttt-3.ipynb   ttt_tactics.py
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16586993255795564277
,
 name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 2011483636964603748
physical_device_desc: "device: XLA_CPU device"
]
tensorflow: 2.3.0
DIR = ./ttt-tactics
MODULE =  ttt_tactics
<function savesamples at 0x7f42d13d6400>
<function loadsamples at 0x7f42d13d6620>
<class 'ttt_tactics.AlphaZeroModel'>
<function AlphaZeroPolicy at 0x7f42d13d68c8>


In [ ]:
### SELF-PLAY ###

model = AlphaZeroModel()
# model.save(f'{DIR}/model-000.h5')
iters = 10_000
while iters > 0:
    iters -= 1

    datas = sortedfiles(f'{DIR}/data-*.tfrec')
    if len(datas) > 50:
        print('removing', datas[0])
        os.remove(datas[0])

    models = sortedfiles(f'{DIR}/model-*.h5')
    model.load(models[-1])

    pi = AlphaZeroPolicy(model, times=100, cp=2, tau=2)
    samples = samplegames({1:pi,2:pi}, iters=100)
    print('samples:', len(samples))

    path = f'{DIR}/data-{round(time())}.tfrec'
    savesamples(path, samples)
    MTIME(path)      

    NOTIFY(f'data ready, len={len(samples)}, path={path}, left={iters}, shell={str(ipython)}', title=ipynb, priority=-1)   


./ttt-tactics/model-008.h5 Oct 09 2020 21:00:05
samplegames 1:searchpi(model-008,times=100,cp=2,tau=2) vs 2:searchpi(model-008,times=100,cp=2,tau=2)
Instructions for updating:
Do not pass `graph_parents`.  They will  no longer be used.
Instructions for updating:
Do not call `graph_parents`.
 14/100 [===>..........................] - ETA: 51:16 - episode: 54.0000 - agent: 0.3571 - rival: 0.3571 - draw: 0.2857